In [1]:
# Import libraries
import numpy as np; # Mostly used for array manipulation datatype numpy.ndarray
from sklearn.linear_model import LinearRegression # Used for implementing Logistic regression
%matplotlib notebook
import matplotlib.pyplot as plt # Plotting in python3 using Matplotlib library
import math # math module for exp function; For e value
# generate random integer values
from numpy.random import seed
from numpy.random import randint
import array
# seed random number generator
seed(1)

In [2]:
def prepare_shapes(num_oflayers,nodes_eachlayer,rows,cols,eachlayer,shape_wts,shape_b):
    for eachlayer in range(num_oflayers):
        ## shape of weights in given current layer is nodesinpreviouslayer X nodesincurrentlayer
        if(eachlayer == 0):
            shape_forwt = (cols,nodes_eachlayer[1])
            shape_wts[str(eachlayer)] = shape_forwt
            
            shape_forb = rows
            shape_b[str(eachlayer)] = shape_forb 
        else:
            shape_forwt = (nodes_eachlayer[eachlayer-1],nodes_eachlayer[eachlayer])
            shape_wts[str(eachlayer)] = shape_forwt
                        
            shape_forb = rows
            shape_b[str(eachlayer)] = shape_forb
    
    
    return shape_wts,shape_b

In [3]:
def lin_reg(features,weights,b):
    lin_reg_output = np.dot(features,weights)+b
    print("shape of input activations:",np.shape(features))
    print("shape of weights:",np.shape(weights))
    print("shape of b:",np.shape(b))
    print("shape of lin reg output:",np.shape(lin_reg_output))

    return lin_reg_output

In [4]:
def logistic_reg(lin_reg_output):
    log_reg_output = 1/(1+np.exp(-lin_reg_output))
    print("shape of logistic reg output:",np.shape(log_reg_output))

    return log_reg_output

In [5]:
def relu_procedure(lin_reg_output):
    #calculate ReLU g(lin_reg or z) = max(0,z) when z>=0 and g(z)=0 when z<0
    print("linear regression output:",lin_reg_output)
    relu_val = lin_reg_output
    relu_val[relu_val<0]=0
    relu_output = relu_val
    print("shape of ReLU output:",np.shape(relu_output))

    return relu_output

In [6]:
def calculate_error(inputs_activations,rows,cols,diagnosis_res,error,lossfunction):
    # error computed from subtracting logistic output & diagnosis 
    log_reg_output = logistic_reg(inputs_activations)
    for eachrow in range(rows):
        error[eachrow] = abs(log_reg_output[eachrow] - diagnosis_res[eachrow])
        lossfunction[eachrow] = (-(diagnosis_res[eachrow])*np.log(log_reg_output[eachrow]))-(1-diagnosis_res[eachrow])*(np.log(1-log_reg_output[eachrow]))
    return error,lossfunction    

In [7]:
def computedjdw(dj_dw,dj_db,error,features,rows,cols):
    for eachrow in range(rows):
        for eachcol in range(cols):
            dj_dw[eachcol] = dj_dw[eachcol] + error[eachrow] + features[eachrow,eachcol]
        dj_db[eachrow] = dj_db[eachrow] + error[eachrow]
    return dj_dw, dj_db

In [8]:
def computecostfunction(lossfunction,rows):
    costfunction = sum(lossfunction)/rows
    return costfunction

In [9]:
def main():
       
    # Input feature 1 patient age
    # Input feature 2 tumor size
    # Input feature 3 family history (y/n:0/1)
    # Input feature 4 regular junk food intake
    # Input feature 5 currently smoking
    # Input feature 6 stressful lifestyle
    # Input feature 7 working with chemicals
    # Input feature 8 urban/rural (0/1)
    # Input feature 9 sleep duration (1hr-10hrs)
    # Input feature 10 regular exercise
    
    num_ofpatients = 398
    pat_age = randint(0,100,num_ofpatients)
    tum_size = randint(1,10,num_ofpatients) # in cm
    fm_histry = randint(0,2,num_ofpatients)
    junk_food = randint(0,2,num_ofpatients)
    smoking = randint(0,2,num_ofpatients)
    stress_lif = randint(0,2,num_ofpatients)
    work_withchem = randint(0,2,num_ofpatients)
    rural_urbn = randint(0,2,num_ofpatients)
    sleep_dur = randint(1,10,num_ofpatients)
    reg_exerc = randint(0,2,num_ofpatients)
    features = np.transpose([pat_age, tum_size, fm_histry, junk_food,smoking,stress_lif,work_withchem,rural_urbn,sleep_dur,reg_exerc]);
    [rows,cols] = np.shape(features)
    
    # Output benign or malignant 0/1
    diagnosis_res = randint(0,2,num_ofpatients)
    
    
        
    print("Enter number of layers:")
    num_oflayers = int(input())
    
    
    
    
    #### Prepare each layer with desired number of nodes ######
    
    nodes_eachlayer = np.empty([num_oflayers],dtype=int)
    
    
    for eachlayer in range(num_oflayers):
       
        if(eachlayer == 0):
            nodes_eachlayer[eachlayer] = cols
        else:
            print("Enter num of nodes for layer no:", eachlayer)
            nodes_eachlayer[eachlayer] = int(input())
    #print(nodes_eachlayer)
    
    
        
    ######## Prepare shapes of weights and bias per each layer #########
    weights = {}
    b = {}
    shape_wts = {}
    shape_b = {}
    shape_wts, shape_b = prepare_shapes(num_oflayers,nodes_eachlayer,rows,cols,eachlayer,shape_wts,shape_b)
    
    print("weight shapes dictionary",shape_wts)
    print("b shapes dictionary",shape_b)
    
    
    ############# No ambiguity w.r.t shapes of weights and bias for each layer ###
    
    dj_dw = {}
    dj_db = {}
    error = np.zeros(rows,dtype = np.float64)
    lossfunction = np.zeros(rows,dtype = np.float64)
    Jcostfunctionold = 0.00
    
    
    max_iterations = 2
    learningratew = 1e-4
    learningrateb = 1e-7
       
    
    print("Input features dim:",np.shape(features))
    
    # initialize weights and b before running ANN, which go into 1st epoch of the ANN
    
    for eachlayer in range(num_oflayers):
        # [numofnodespreviouslayer,numofnodescurrlayer] = 
        numofnodespreviouslayer = shape_wts[str(eachlayer)][0]
        numofnodescurrlayer = shape_wts[str(eachlayer)][1]
        weights[str(eachlayer)] = randint(1,10,size = (numofnodespreviouslayer,numofnodescurrlayer))  # np.zeros(shape_wts[str(eachlayer)],dtype=np.float64) 
        b[str(eachlayer)] = randint(1,100,size = shape_b[str(eachlayer)]).reshape(rows,1) # np.zeros(shape_b[str(eachlayer)],dtype=np.float64) 
        
        dj_dw = np.zeros(cols,dtype=np.float64)
        dj_db = np.zeros(shape_b[str(eachlayer)],dtype=np.float64)
        
        
    ###### Finish initialization of weights, b for epoch 0
    
    #print("Initialized weights",weights)
    #print("Initialized b",b)
    
    Jcostfunctionold = 1000
    
    for eachepoch in range(max_iterations):
        print("Epoch No:",eachepoch)
        for eachlayer in range(num_oflayers):
            if(eachlayer == 0):
                inputs_activations = features
            
            else:
                print("layer no:",eachlayer)
                ## Since this is a single hidden layer ANN, only (Hidden) (index:layer 1)
                # Linear OR logistic OR softmax OR ReLU is done
                lin_reg_output = lin_reg(inputs_activations,weights[str(eachlayer)],b[str(eachlayer)])
                log_reg_output = logistic_reg(lin_reg_output)
                relu_output = relu_procedure(lin_reg_output)
                inputs_activations = lin_reg_output
                #softmax_output = softmax_reg()
    
        if(num_oflayers==3): # layer 0: input layer, layer 1: hidden layer, layer 2: output layer
            error = np.zeros(rows,dtype = float)
            lossfunction = np.zeros(rows,dtype = float)
            error,lossfunction = calculate_error(inputs_activations,rows,cols,diagnosis_res,error,lossfunction)
            dj_dw,dj_db = computedjdw(dj_dw,dj_db,error,features,rows,cols)
            costfunction = computecostfunction(lossfunction,rows)
            
            Jcostfunctionnew = costfunction
            if(Jcostfunctionold<Jcostfunctionnew):
                break
            else:
                print("Costfunction value:",Jcostfunctionold)
                Jcostfunctionold = Jcostfunctionnew
     
    
    
    
    

if __name__ == "__main__":
    main()  

Enter number of layers:
3
Enter num of nodes for layer no: 1
23
Enter num of nodes for layer no: 2
1
weight shapes dictionary {'0': (10, 23), '1': (10, 23), '2': (23, 1)}
b shapes dictionary {'0': 398, '1': 398, '2': 398}
Input features dim: (398, 10)
Epoch No: 0
layer no: 1
shape of input activations: (398, 10)
shape of weights: (10, 23)
shape of b: (398, 1)
shape of lin reg output: (398, 23)
shape of logistic reg output: (398, 23)
linear regression output: [[445 251 456 ... 181 269 477]
 [180 120 185 ...  89 147 215]
 [733 421 747 ... 223 345 769]
 ...
 [340 168 342 ...  89 140 349]
 [187 109 206 ...  71 121 213]
 [299 148 318 ...  88 153 320]]
shape of ReLU output: (398, 23)
layer no: 2
shape of input activations: (398, 23)
shape of weights: (23, 1)
shape of b: (398, 1)
shape of lin reg output: (398, 1)
shape of logistic reg output: (398, 1)
linear regression output: [[43346]
 [19717]
 [66566]
 [19935]
 [59239]
 [18923]
 [71234]
 [46900]
 [20359]
 [19515]
 [61860]
 [66635]
 [14563]


/tmp/ipykernel_27537/461491038.py:6: RuntimeWarning: divide by zero encountered in log
  lossfunction[eachrow] = (-(diagnosis_res[eachrow])*np.log(log_reg_output[eachrow]))-(1-diagnosis_res[eachrow])*(np.log(1-log_reg_output[eachrow]))
/tmp/ipykernel_27537/461491038.py:6: RuntimeWarning: invalid value encountered in multiply
  lossfunction[eachrow] = (-(diagnosis_res[eachrow])*np.log(log_reg_output[eachrow]))-(1-diagnosis_res[eachrow])*(np.log(1-log_reg_output[eachrow]))
